In [5]:
using Optim
Optim.UnconstrainedProblems.examples["Rosenbrock"]
# These codes turn f and g into one function ... 
function opt_combine(x, f, g!)
    g = Vector{Float64}(length(x))
    g!(x,g)
    return (f(x), g)
end
function opt_problem(p::Optim.UnconstrainedProblems.OptimizationProblem)
    return x -> opt_combine(x, p.f, p.g!)
end
# this just makes it easy to use
opt_problem(s::AbstractString) = opt_problem(
    Optim.UnconstrainedProblems.examples[s])

# Here's an example 
fg = opt_problem("Himmelblau")
f, g = fg([0.0,0.0]) # show the function and gradient

(170.0,[-14.0,-22.0])

In [6]:
problems = keys(Optim.UnconstrainedProblems.examples)

Base.KeyIterator for a Dict{String,Optim.UnconstrainedProblems.OptimizationProblem} with 9 entries. Keys:
  "Exponential"
  "Fletcher-Powell"
  "Rosenbrock"
  "Parabola"
  "Hosaki"
  "Large Polynomial"
  "Polynomial"
  "Powell"
  "Himmelblau"

In [58]:
using Iterators
using Plots
plotlyjs()
"""
Make a line search plot that shows what the function
looks like in a direction p, with regions for the
strong Wolfe conditions satisfied. 
"""
function line_search_plot(fg, x, p; amax=1.0, c2 = 0.90, c1 = 0.1, npts=1000, strong=false)
    agrid = linspace(0,amax,npts)
    fvals = map(a -> fg(x + a*p)[1], agrid) # generate function values
    plt = plot(agrid, fvals, label="f")
    
    g0 = vecdot(fg(x)[2],p)
    f0 = fg(x)[1]
    
    plot!([agrid[1], agrid[end]],[f0,f0+c1*g0*agrid[end]],label="c1")
        
    gvals = map(a -> abs(vecdot(fg(x+a*p)[2],p)), agrid)
    
    if strong
        validwolfe = a -> (abs(vecdot(fg(x+a*p)[2],p)) <= c2*abs(g0)) * 
                          (fg(x + a*p)[1] <= f0+c1*g0*a)
    else
        validwolfe = a -> (vecdot(fg(x+a*p)[2],p) >= c2*g0) * 
                          (fg(x + a*p)[1] <= f0+c1*g0*a)
    end
    
    
    regions = groupby(validwolfe, agrid)
    for r in regions
        r1 = r[1]
        @show r1, r[end]
        
        if validwolfe(r1)
            plot!([r[1],r[end]], [f0, f0],fill=(0,:green,0.5),line=(0),label="c2")
        end
    end
    
    
    #gmax = abs(g0)*c2
    
    # find conseq regions where gvals >= gmax
    
    
    
    #c1=plot!([0.1, 1.0], [f0, f0],fill=(0,:green,0.5),label="c3")
#append!(p, 1, [1.0,1.6,1.6,1.8], [0,0,1,1])
#append!(p, 1, [1.8,2.2,2.2,2.5], [0,0,1,1])
#append!(p, 1, [2.5,3.2,3.2,3.5], [0,0,1,1])
   
    return plt
end
fg = opt_problem("Rosenbrock")
x = [0.0,0.0]
p = [1.0,0.0]
line_search_plot(fg, x, p; strong=true)



(r1,r[end]) = (0.0,0.05805805805805806)
(r1,r[end]) = (0.05905905905905906,0.2032032032032032)
(r1,r[end]) = (0.2042042042042042,1.0)


In [64]:
fg = opt_problem("Himmelblau")
x = [0.0,0.0]
p = [1.0,0.0]
line_search_plot(fg, x, p; strong=false, amax=6)

(r1,r[end]) = (0.0,3.255255255255255)
(r1,r[end]) = (3.2612612612612613,4.852852852852853)
(r1,r[end]) = (4.858858858858859,6.0)


In [65]:
fg = opt_problem("Himmelblau")
x = [0.0,0.0]
p = [1.0,0.0]
line_search_plot(fg, x, p; strong=true, amax=6)

(r1,r[end]) = (0.0,3.255255255255255)
(r1,r[end]) = (3.2612612612612613,3.5195195195195197)
(r1,r[end]) = (3.5255255255255253,6.0)


In [77]:
using Plots
plotlyjs()
ezcontour(x, y, f) = begin
    X = repmat(x', length(y), 1)
    Y = repmat(y, 1, length(x))
    # Evaluate each f(x, y)
    Z = map((x,y) -> log(f([x,y])), X, Y)
    plot(x, y, Z, st=:contour)
end

function line_search_contour(fg, x, y, xk, p; amax=1.0)
    ezcontour(x, y, z -> fg(z)[1])
    scatter!([xk[1]],[xk[2]],label="")
    plot!([xk[1], xk[1] + amax*p[1]], [xk[2], xk[2] + amax*p[2]], label="")
end

line_search_contour(fg, -6:0.05:6, -6:0.05:6, x, p; amax=6.0)

In [ ]:
# TODO - make an interact module that allows you to vary distances in a 2d region and look at line searches! 